Цель задания: Исследование алгоритмов решения задач методом поиска.
Описание предметной области. Имеется транспортная сеть, связывающая
города СНГ. Сеть представлена в виде таблицы связей между городами. Связи
являются двусторонними, т.е. допускают движение в обоих направлениях.
Необходимо проложить маршрут из одной заданной точки в другую.
Этап 1. Неинформированный поиск. На этом этапе известна только
топология связей между городами.
Выполнить:
1)поиск в ширину;
2)поиск глубину;
3)поиск с ограничением глубины;
4)поиск с итеративным углублением;
5)двунаправленный поиск.
Отобразить движение по дереву на его графе с указанием сложности каждого вида поиска. Сделать выводы.


In [58]:
import networkx as nx
from networkx import *
import pylab as plt
import os
import collections

In [59]:
table = '''Вильнюс Брест 531
Витебск Брест 638
Витебск Вильнюс 360
Воронеж Витебск 869
Воронеж Волгоград 581
Волгоград Витебск 1455
Витебск Ниж.Новгород 911
Вильнюс Даугавпилс 211
Калининград Брест 699
Калининград Вильнюс 333
Каунас Вильнюс 102
Киев Вильнюс 734
Киев Житомир 131
Житомир Донецк 863
Житомир Волгоград 1493
Кишинев Киев 467
Кишинев Донецк 812
С.Петербург Витебск 602
С.Петербург Калининград 739
С.Петербург Рига 641
Москва Казань 815
Москва Ниж.Новгород 411
Москва Минск 690
Москва Донецк 1084
Москва С.Петербург 664
Мурманск С.Петербург 1412
Мурманск Минск 2238
Орел Витебск 522
Орел Донецк 709
Орел Москва 368
Одесса Киев 487
Рига Каунас 267
Таллинн Рига 308
Харьков Киев 471
Харьков Симферополь 639
Ярославль Воронеж 739
Ярославль Минск 940
Уфа Казань 525
Уфа Самара 461'''
# изначальные данные
table

'Вильнюс Брест 531\nВитебск Брест 638\nВитебск Вильнюс 360\nВоронеж Витебск 869\nВоронеж Волгоград 581\nВолгоград Витебск 1455\nВитебск Ниж.Новгород 911\nВильнюс Даугавпилс 211\nКалининград Брест 699\nКалининград Вильнюс 333\nКаунас Вильнюс 102\nКиев Вильнюс 734\nКиев Житомир 131\nЖитомир Донецк 863\nЖитомир Волгоград 1493\nКишинев Киев 467\nКишинев Донецк 812\nС.Петербург Витебск 602\nС.Петербург Калининград 739\nС.Петербург Рига 641\nМосква Казань 815\nМосква Ниж.Новгород 411\nМосква Минск 690\nМосква Донецк 1084\nМосква С.Петербург 664\nМурманск С.Петербург 1412\nМурманск Минск 2238\nОрел Витебск 522\nОрел Донецк 709\nОрел Москва 368\nОдесса Киев 487\nРига Каунас 267\nТаллинн Рига 308\nХарьков Киев 471\nХарьков Симферополь 639\nЯрославль Воронеж 739\nЯрославль Минск 940\nУфа Казань 525\nУфа Самара 461'

In [60]:
graph = read_edgelist(r"data/data_city.txt") #считываем граф из файла, без веса

In [61]:
graph = nx.Graph(graph)

In [62]:
nx.draw(graph,  with_labels=True, pos=nx.circular_layout(graph))
plt.savefig('graph.png')
plt.close()
nx.draw(graph,  with_labels=True, pos=nx.kamada_kawai_layout(graph))
plt.savefig('graph_2.png')
plt.close()

![](graph.png)
![](graph_2.png)


In [63]:
table = list(map(lambda x: x[0:2] + [int(x[2])], map(lambda x: x.split(), table.split('\n'))))
#разбиваем на списки смежности данные с учетом веса
table

[['Вильнюс', 'Брест', 531],
 ['Витебск', 'Брест', 638],
 ['Витебск', 'Вильнюс', 360],
 ['Воронеж', 'Витебск', 869],
 ['Воронеж', 'Волгоград', 581],
 ['Волгоград', 'Витебск', 1455],
 ['Витебск', 'Ниж.Новгород', 911],
 ['Вильнюс', 'Даугавпилс', 211],
 ['Калининград', 'Брест', 699],
 ['Калининград', 'Вильнюс', 333],
 ['Каунас', 'Вильнюс', 102],
 ['Киев', 'Вильнюс', 734],
 ['Киев', 'Житомир', 131],
 ['Житомир', 'Донецк', 863],
 ['Житомир', 'Волгоград', 1493],
 ['Кишинев', 'Киев', 467],
 ['Кишинев', 'Донецк', 812],
 ['С.Петербург', 'Витебск', 602],
 ['С.Петербург', 'Калининград', 739],
 ['С.Петербург', 'Рига', 641],
 ['Москва', 'Казань', 815],
 ['Москва', 'Ниж.Новгород', 411],
 ['Москва', 'Минск', 690],
 ['Москва', 'Донецк', 1084],
 ['Москва', 'С.Петербург', 664],
 ['Мурманск', 'С.Петербург', 1412],
 ['Мурманск', 'Минск', 2238],
 ['Орел', 'Витебск', 522],
 ['Орел', 'Донецк', 709],
 ['Орел', 'Москва', 368],
 ['Одесса', 'Киев', 487],
 ['Рига', 'Каунас', 267],
 ['Таллинн', 'Рига', 308],
 ['Хар

In [64]:
#Мой вариант 7
START = 'Рига'
END = 'Одесса'
a_list = table
a_dict = {el[0]:dict() for el in a_list}
for el in a_list:
    a_dict[el[1]] = dict()
print(a_dict)
for el in a_list:
    a_dict[el[0]][el[1]] = el[2]
    a_dict[el[1]][el[0]] = el[2]
a_dict # вложенный словарь, с которым будем работать

{'Вильнюс': {}, 'Витебск': {}, 'Воронеж': {}, 'Волгоград': {}, 'Калининград': {}, 'Каунас': {}, 'Киев': {}, 'Житомир': {}, 'Кишинев': {}, 'С.Петербург': {}, 'Москва': {}, 'Мурманск': {}, 'Орел': {}, 'Одесса': {}, 'Рига': {}, 'Таллинн': {}, 'Харьков': {}, 'Ярославль': {}, 'Уфа': {}, 'Брест': {}, 'Ниж.Новгород': {}, 'Даугавпилс': {}, 'Донецк': {}, 'Казань': {}, 'Минск': {}, 'Симферополь': {}, 'Самара': {}}


{'Вильнюс': {'Брест': 531,
  'Витебск': 360,
  'Даугавпилс': 211,
  'Калининград': 333,
  'Каунас': 102,
  'Киев': 734},
 'Витебск': {'Брест': 638,
  'Вильнюс': 360,
  'Воронеж': 869,
  'Волгоград': 1455,
  'Ниж.Новгород': 911,
  'С.Петербург': 602,
  'Орел': 522},
 'Воронеж': {'Витебск': 869, 'Волгоград': 581, 'Ярославль': 739},
 'Волгоград': {'Воронеж': 581, 'Витебск': 1455, 'Житомир': 1493},
 'Калининград': {'Брест': 699, 'Вильнюс': 333, 'С.Петербург': 739},
 'Каунас': {'Вильнюс': 102, 'Рига': 267},
 'Киев': {'Вильнюс': 734,
  'Житомир': 131,
  'Кишинев': 467,
  'Одесса': 487,
  'Харьков': 471},
 'Житомир': {'Киев': 131, 'Донецк': 863, 'Волгоград': 1493},
 'Кишинев': {'Киев': 467, 'Донецк': 812},
 'С.Петербург': {'Витебск': 602,
  'Калининград': 739,
  'Рига': 641,
  'Москва': 664,
  'Мурманск': 1412},
 'Москва': {'Казань': 815,
  'Ниж.Новгород': 411,
  'Минск': 690,
  'Донецк': 1084,
  'С.Петербург': 664,
  'Орел': 368},
 'Мурманск': {'С.Петербург': 1412, 'Минск': 2238},
 'Орел': {

In [65]:
class Queue:
    def __init__(self):
        self.elements = collections.deque()

    def empty(self):
        return len(self.elements) == 0

    def put(self, x):
        self.elements.append(x)

    def get(self):
        return self.elements.popleft()

# Поиск в ширину

In [66]:
def breadth_first_search_1(graph, start, end): #Трудоемкость O(N+M) рассматриваем все ребра и вершины единожды
    # печать того, что мы нашли
    frontier = Queue() # очередь
    frontier.put(start) # начинаем с вершины, которую передали в start
    visited = {} # пустой словарь вершин, которые посетили
    visited[start] = True
    parents = {}
    parents[start] = None

    while not frontier.empty(): #пока очередь не пуста
        current = frontier.get() #берем элемент
     #   print("Visiting %r" % current) выводит обход графа в ширину
        for next in graph.neighbors(current): #смотрим всех соседей
            if next not in visited:
                frontier.put(next)
                parents[next] = current #записываем родителя для каждой вершины
                visited[next] = True

    path = [end]
    parent = parents[end]
    while not parent is None:
        path.append(parent)
        parent = parents[parent]
    for x in  path[::-1]:
        print(x, ('---->' if x!=end else ' поиск в ширину '), end=' ', )



breadth_first_search_1(graph, START, END) # использую graph из библиотеки networkx

Рига ----> Каунас ----> Вильнюс ----> Киев ----> Одесса  поиск в ширину  

# Поиск в глубину



In [72]:
parents = dict()
parents[START]=None
def dfs(graph, start, end, visited = None):  #function for dfs
    visited = visited or set() # в python при or если одно из выражений True, то сразу принимает это значение
    visited.add(start)
    for neighbour in graph[start]:
        if neighbour not in visited:
            parents[neighbour] = start
            dfs(graph, neighbour, end, visited)


dfs(a_dict, START, END, {}) # здесь использую вложенный словарь, который создал в самом начале
path=[END]
parent = parents[END]
while not parent is None:
    path.append(parent)
    parent = parents[parent]
for x in  path[::-1]:
    print(x, ('---->' if x!=END else ' поиск в глубину '), end=' ', )

Рига ----> С.Петербург ----> Витебск ----> Брест ----> Вильнюс ----> Киев ----> Одесса  поиск в глубину  

# Поиск с ограничением глубины

# Поиск с итеративным углублением